# Seq2seq으로 한영 번역기 만들기(Kor2Eng)

- 한국어 문장을 입력하면 영어로 번역된 문장을 출력하는 번역기를 만들어 봅니다.
- Sequence-to-Sequence 구조는 두 개의 RNN 모듈을 Encoder-Decoder 구조로 결합하여 사용하는 구조로 번역기에 최적화 되어 있습니다.
- 이번 프로젝트에서는 Seq2seq 기반 번역기를 직접 만들어보며 그 구조를 이해해보고 또한 Attention 기법을 추가하여 성능을 높여 볼 것입니다.

한국어를 잘 시각화 하기 위해서 한국어 폰트를 다운로드 합니다.
- 설치후 LMS를 재실행해야 설치된 글꼴이 제대로 보입니다
- matplotlib 라이브러리의 기본 폰트는 한국어를 지원하지 않습니다

In [1]:
!sudo apt -qq -y install fonts-nanum

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


사용할 라이브러리들을 불러옵니다.

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [3]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.4.1


## 데이터 준비
[jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)

한영 병렬 데이터 korean-english-park.train.tar.gz를 다운받습니다.
- 94123개의 한국어-영어 paired data가 존재합니다.

In [4]:
cache_dir = '~/aiffel/going_deeper/gd8'
path_to_zip = tf.keras.utils.get_file(
    'korean-english-park.train.tar.gz',
    origin = 'https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz',
    cache_dir = cache_dir,
    extract=True
    )

8724480/8718893 [==============================] - 0s 0us/step


In [7]:
path_to_en = os.path.dirname(path_to_zip)+"/korean-english-park.train.en"
path_to_ko = os.path.dirname(path_to_zip)+"/korean-english-park.train.ko"

In [15]:
with open(path_to_ko, "r") as f:
    raw_ko=f.read().splitlines()
with open(path_to_en, "r") as f:
    raw_en=f.read().splitlines()
    
print(f"Data Size: ko_{len(raw_ko)}, en_{len(raw_en)}")
print("Example:")

for sen_ko, sen_en in list(zip(raw_ko, raw_en))[:5]:
    print(">>\n",sen_ko, "\n",sen_en)

Data Size: ko_94123, en_94123
Example:
>>
 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?" 
 Much of personal computing is about "can you top this?"
>>
 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다. 
 so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.
>>
 그러나 이것은 또한 책상도 필요로 하지 않는다. 
 Like all optical mice, But it also doesn't need a desk.
>>
 79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다. 
 uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.
>>
 정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다. 
 Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack.


## 데이터 정제

### 중복데이터 제거

set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거하도록 합니다. 데이터의 병렬 쌍이 흐트러지지 않도록 주의해야 합니다.중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.

In [20]:
cleaned_corpus = list(set(zip(raw_ko, raw_en)))
len(cleaned_corpus)

78968

In [22]:
cleaned_corpus[:3]

[('미 캘리포니아 글렌데일에 거주하는 라보프는 오는 11월 28일 쿡카운티 지방 법원에 출석해야 한다.',
  'LaBeouf, of Glendale, California, is scheduled to appear in Cook County court on November 28.'),
 ('일본에서 16일(현지시각) 기상관측 이래 최고 기온이 기록돼 폭염으로 7명이 목숨을 잃었다.',
  'TOKYO, Japan (CNN) Temperatures hit record highs in Japan on Thursday as a heat wave swept through the country, leaving at least seven people dead over the last few days.'),
 ('미국은 2만1천5백 여명의 미군을 이라크에 더 보낼 예정이라고 발표한 바 있다.',
  'Bush recently announced plans to send 21,500 more US troops to Iraq.')]

### 데이터 전처리: 정제하기
- 불필요한 노이즈로 작용할 수 있는 특수문자는 정제과정에서 제거합니다.
- 전처리 과정에서 문장의 시작문자 , 종료문자 를 붙여줍니다. Decoder는 첫 입력으로 사용할 시작 토큰과 문장생성 종료를 알리는 끝 토큰이 반드시 필요하기 때문에 Decoder에게 필수적인 작업입니다.

In [54]:
def preprocess_sentence(sentence, s_token=False, e_token=False):
    
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence) #단어와 구두점 사이의 공백 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 여러개 하나의 공백으로 치환
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9?.!,]+", " ", sentence) #a-zA-Z?.!, 제외한 문자 제거
    
    sentence = sentence.strip() # 양 끝 공백 제거
    
    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [55]:
enc_corpus = []
dec_corpus = []

for pair in cleaned_corpus:
    ko, en = pair
    
    enc_corpus.append(preprocess_sentence(ko))
    dec_corpus.append(preprocess_sentence(en, s_token=True, e_token=True))

print("한국어:", enc_corpus[20])
print("영어:", dec_corpus[20])

한국어: 그들은 자신들의 요구조건이 수용되지 않을 경우 인질을 추가 살해하겠다고 경고하기도 했다 .
영어: <start> The Taliban killed two male hostages and have long said they would kill others unless their demands were met . <end>


### 데이터 전처리: 토큰화
- 한글 토큰화는 KoNLPy의 mecab 클래스를 사용합니다.
- 모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸립니다. cleaned_corpus로부터 토큰의 길이가 40 이하인 데이터를 선별하여 eng_corpus와 kor_corpus를 구축합니다.
- tokenize() 함수를 사용해 데이터를 텐서로 변환하고 각각의 tokenizer를 얻습니다. 단어의 수는 실험을 통해 적당한 값을 맞춰주도록 합니다! (최소 10,000 이상)
- 난이도에 비해 데이터가 많지 않아 훈련 데이터와 검증 데이터를 따로 나누지는 않습니다.

In [28]:
#- Konlpy Mecab 설치 
# Install dependencies
! sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
# Install KoNLPy
! python3 -m pip install --upgrade pip
! python3 -m pip install konlpy  # Python 3.x
# Install MeCab (Optional)
! sudo apt-get install curl git 
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
openjdk-8-jdk is already the newest version (8u282-b08-0ubuntu1~18.04).
The following additional packages will be installed:
  dh-python libexpat1-dev libpython3-dev libpython3.6-dev python-pip-whl
  python3-asn1crypto python3-cffi-backend python3-crypto python3-cryptography
  python3-distutils python3-keyring python3-keyrings.alt python3-lib2to3
  python3-secretstorage python3-setuptools python3-wheel python3-xdg
  python3.6-dev
Suggested packages:
  python-crypto-doc python-cryptography-doc python3-cryptography-vectors
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0
  python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  dh-python libexpat1-dev libpython3-dev libpython3.6-dev python-pip-whl
  python3-asn1crypto python3-cffi-backe

In [42]:
from konlpy.tag import Mecab
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:
def tokenize(corpus, vocab_size=30000, max_len=40, padding='post'):
    tokenizer=Mecab()
    
    X_train = []
    for sentence in corpus:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        X_train.append(temp_X)
        
        
    # 단어사전 만들기
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(vocab_size-4)  # 단어 빈도순으로 vocab_size 만큼 가져오기
    vocab = ['<PAD>','<UNK>','<start>','<end>'] + [key for key, _ in counter]  # 앞부분 4개 추가
    word_to_index = {word:index for index, word in enumerate(vocab)}  # {단어:인덱스} 단어사전 생성
    
    # 텍스트를 단어사전 인덱스로 변환
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
    
    X_train = list(map(wordlist_to_indexlist, X_train))
    
    # index to word
    index_to_word = {index:word for word, index in word_to_index.items()}
    
    # padding으로 문장 길이 맞추기
    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                       value=word_to_index["<PAD>"],
                                       padding=padding,
                                       maxlen=max_len)
    
    return X_train, word_to_index

In [ ]:
# 토큰화
enc_tensor, enc_tokenizer = tokenize(enc_corpus, vocab_size = 30000, max_len=40, padding='post')
dec_tensor, dec_tokenizer = tokenize(dec_corpus, vocab_size = 30000+2, max_len=40, padding='post')

In [ ]:
print("Korean Vocab Size:", len(enc_tokenizer))
print("English Vocab Size:", len(dec_tokenizer))

## 모델 설계
- Attention 기반 Seq2seq 모델을 설계합니다.
    - 인코더-디코더에 GRU 1개씩 갖는 구조
    - 인코더는 모든 Time-Step의 Hidden State를 출력으로 갖고, 디코더는 인코더의 출력과 t-1의 Step의 Hidden State로 Attention을 취하여 t Step의 Hidden State를 생성
    - 디코더에서 t Step의 단어로 예측된 것을 실제 정답과 대조해 Loss를 구하고, 생성된 t Step의 Hidden State는 t+1 Step의 Hidden State를 만들기 위해 다시 Decoder에 전달
    - t=1 일 때의 Hidden State는 일반적으로 Encoder의 Final State를 Hidden State로 사용
    - Attention은 Bahdanau을 사용
- Dropout 모듈을 추가하여 성능을 향상시켜 봅니다.
- Embedding Size와 Hidden Size는 실험을 통해 적당한 값을 맞춰줍니다.

In [58]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)
        
        # score shape == (batch_size, max_length, 1)
        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        # attention weights shape == (batch_size, max_length, 1)
        attn = tf.nn.softmax(score, axis=1)
        
        # context vector shape after sum == (batch_size, hidden_size)
        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [59]:
# Encoder
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units, dropout = 0.3, return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

# Decoder
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       dropout = 0.3,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        self.attention = BahdanauAttention(self.dec_units)   # Attention 필수 사용!

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)
        
        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)
        
        return out, h_dec, attn

In [64]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(enc_tokenizer) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

In [65]:
# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (64, 30, 1024)
Decoder Output: (64, 30003)
Decoder Hidden State: (64, 1024)
Attention: (64, 30, 1)


## 훈련하기

### Optimizer & Loss

In [67]:
optimizer=tf.keras.optimizers.Adam()
loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask=tf.math.logical_not(tf.math.equal(real,0))
    loss=loss_object(real,pred)
    
    mask=tf.cast(mask, dtype=loss.dtype)
    loss*=mask
    
    return tf.reduce_mean(loss)

### Train step

In [74]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [75]:
from tqdm import tqdm
import random

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)    # tqdm

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_tensor[idx:idx+BATCH_SIZE],
                                dec_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))    # tqdm
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))    # tqdm

Epoch  1:  22%|██▏       | 271/1234 [07:55<28:10,  1.76s/it, Loss 5.0393] 


KeyboardInterrupt: 

## 평가하기

- 아래 예문을 번역해봅니다.

  -  K1) 오바마는 대통령이다. <br>
  -  K2) 시민들은 도시 속에 산다.<br>
  -  K3) 커피는 필요 없다.<br>
  -  K4) 일곱 명의 사망자가 발생했다.<br>
    
- Attention Map을 시각화합니다.

In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))

In [ ]:
translate("Can I have some coffee?", encoder, decoder)

## 루브릭
|평가문항|상세기준|
|:---|:---|
|1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.|구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.|
|2. Attentional Seq2seq 모델이 정상적으로 구동된다.|seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.|
|3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.|